In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import shutil
import os

src_dir = '/content/drive/MyDrive/facial_fuse/label-distribution-learning-fer-tf'
dst_dir = '/content/facial_fuse/label-distribution-learning-fer-tf'

os.makedirs(os.path.dirname(dst_dir), exist_ok=True)

shutil.copytree(src_dir, dst_dir, dirs_exist_ok=True)

print(" Folder copied to /content/facial_fuse/label-distribution-learning-fer-tf")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/facial_fuse/label-distribution-learning-fer-tf'

In [2]:
import pickle
import numpy as np
from PIL import Image

with open('/content/drive/MyDrive/facial_fuse/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('/content/drive/MyDrive/facial_fuse/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)


print(f" Loaded {len(train_data)} training samples.")
print(f" Loaded {len(test_data)} testing samples.")
print(" First train sample keys:", train_data[0].keys())


sample = train_data[0]
print("\n Sample Structure:")
for k, v in sample.items():
    print(f"{k}: {v}")

import numpy as np
landmark = np.load(sample['npy'])
print(f"\n Landmark shape: {landmark.shape}")

img = Image.open(sample['img']).convert('RGB')
img_array = np.array(img)
print(f" Image shape: {img_array.shape}")  #(H, W, 3)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/facial_fuse/train_data.pkl'

In [3]:
import tensorflow as tf
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
mlp_backbone = tf.keras.models.load_model('/content/facial_fuse/label-distribution-learning-fer-tf/pretrained/mlp_landmark_backbone.h5')
resnet_backbone = tf.keras.models.load_model('/content/facial_fuse/label-distribution-learning-fer-tf/pretrained/resnet50.h5')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/facial_fuse/label-distribution-learning-fer-tf/pretrained/mlp_landmark_backbone.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# resnetonly_plain = tf.keras.models.load_model('/content/drive/MyDrive/facial_fuse/resnetonly_plain.h5')
# latefusion_plain = tf.keras.models.load_model('/content/drive/MyDrive/facial_fuse/latefusion_plain.h5')
# conf_head_only = tf.keras.models.load_model('/content/drive/MyDrive/facial_fuse/conf_head_only.h5')
# contra_loss_only = tf.keras.models.load_model('/content/drive/MyDrive/facial_fuse/contra_loss_only.h5')
# full_model = tf.keras.models.load_model('/content/drive/MyDrive/facial_fuse/full_model.h5')

In [4]:
import tensorflow as tf
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

resnet_backbone = tf.keras.applications.ResNet50(
    include_top=False, pooling='avg', input_shape=(112, 112, 3)
)
resnet_backbone.trainable = False

class ImageOnlyModel(tf.keras.Model):
    def __init__(self, image_backbone=None, num_classes=7):
        super().__init__()
        self.image_backbone = image_backbone or tf.keras.applications.ResNet50(
            include_top=False, pooling='avg', input_shape=(112, 112, 3)
        )
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

    def call(self, image_input, training=False, return_features=False):
        img_feat = self.image_backbone(image_input, training=training)
        if return_features:
            return img_feat
        return self.classifier(img_feat, training=training)

    def get_config(self):
        return {}

    @classmethod
    def from_config(cls, config):
        return cls()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [5]:
import h5py

file_path = '/content/drive/MyDrive/facial_fuse/resnetonly_plain.h5'
with h5py.File(file_path, 'r') as f:
    print("🔍 Top-level keys:", list(f.keys()))

    if 'model_weights' in f:
        print("\n This file contains full model weights.")
        print(" Layer groups inside 'model_weights':")
        for layer_name in f['model_weights']:
            print("  ", layer_name)

    if 'optimizer_weights' in f:
        print("\n🛠 Optimizer weights found — this was saved from a trained model with optimizer.")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/facial_fuse/resnetonly_plain.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [6]:
import tensorflow as tf
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

resnet_backbone = tf.keras.applications.ResNet50(
    include_top=False, pooling='avg', input_shape=(112, 112, 3)
)
resnet_backbone.trainable = False

class ImageOnlyModel(tf.keras.Model):
    def __init__(self, image_backbone=None, num_classes=7):
        super().__init__()
        self.image_backbone = image_backbone or tf.keras.applications.ResNet50(
            include_top=False, pooling='avg', input_shape=(112, 112, 3)
        )
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

    def call(self, image_input, training=False, return_features=False):
        img_feat = self.image_backbone(image_input, training=training)
        if return_features:
            return img_feat
        return self.classifier(img_feat, training=training)

    def get_config(self):
        return {}

    @classmethod
    def from_config(cls, config):
        return cls()

model = ImageOnlyModel(resnet_backbone)
model.build(input_shape=(None, 3, 112, 112))


model.load_weights('/content/drive/MyDrive/facial_fuse/resnetonly_plain.h5')


def preprocess_image_only(sample):
    img = Image.open(sample['img']).convert('RGB').resize((112, 112))
    img_arr = np.array(img).astype(np.float32)
    img_arr = (img_arr - 127.5) / 127.5
    img_arr = np.transpose(img_arr, (2, 0, 1))  # (3, 112, 112)
    label = sample['label']
    return img_arr, label

def image_only_generator(data_list):
    for sample in data_list:
        img, label = preprocess_image_only(sample)
        yield img, label

test_dataset = tf.data.Dataset.from_generator(
    lambda: image_only_generator(test_data),
    output_signature=(
        tf.TensorSpec(shape=(3, 112, 112), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
).batch(32).prefetch(tf.data.AUTOTUNE)

class_names = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger']

all_preds, all_labels, all_logits, all_features = [], [], [], []

for batch in test_dataset:
    images, labels = batch
    logits = model(images, training=False)
    features = model(images, training=False, return_features=True)

    preds = tf.argmax(logits, axis=1)

    all_preds.extend(preds.numpy())
    all_labels.extend(labels.numpy())
    all_logits.extend(logits.numpy())
    all_features.extend(features.numpy())

all_labels_np = np.array(all_labels)
all_preds_np = np.array(all_preds)
all_logits_np = np.array(all_logits)

acc = accuracy_score(all_labels_np, all_preds_np)


ce_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
cross_entropy = ce_loss_fn(all_labels_np, all_logits_np).numpy()

uniform_probs = np.full_like(all_logits_np, 1.0 / all_logits_np.shape[1])
kl_div_fn = tf.keras.losses.KLDivergence()
kl_div = kl_div_fn(uniform_probs, all_logits_np).numpy()

conf_matrix = confusion_matrix(all_labels_np, all_preds_np, labels=np.arange(len(class_names)))
per_class_acc = conf_matrix.diagonal() / np.maximum(conf_matrix.sum(axis=1), 1)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Emotion')
plt.ylabel('True Emotion')
plt.gca().invert_yaxis()
plt.show()

tsne = TSNE(n_components=2, random_state=42)
tsne_proj = tsne.fit_transform(np.array(all_features))

plt.figure(figsize=(8, 6))
scatter = plt.scatter(tsne_proj[:, 0], tsne_proj[:, 1], c=all_labels_np, cmap='tab10', alpha=0.6)
handles, _ = scatter.legend_elements()
plt.legend(handles, class_names, title='Emotions')
plt.title('t-SNE of Backbone Features')
plt.show()

print(f"Top-1 Accuracy: {acc:.4f}")
print(f"Cross-Entropy Loss: {cross_entropy:.4f}")
print(f"KL Divergence (vs uniform): {kl_div:.4f}")
print("Per-class Accuracy:")
for i, a in enumerate(per_class_acc):
    print(f"  {class_names[i]}: {a:.4f}")

print("\n Classification Report:")
print(classification_report(all_labels_np, all_preds_np, target_names=class_names, digits=4))


print("\n Label distribution in test data:")
(unique, counts) = np.unique(all_labels_np, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  {class_names[u]} ({u}): {c} samples")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'image_only_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/facial_fuse/resnetonly_plain.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [7]:
import tensorflow as tf
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

class_names = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger']

mlp_backbone = tf.keras.models.load_model('/content/facial_fuse/label-distribution-learning-fer-tf/pretrained/mlp_landmark_backbone.h5')
mlp_backbone.trainable = False
resnet_backbone.trainable = False

class LateFusionModel(tf.keras.Model):
    def __init__(self, image_backbone, lmk_backbone, num_classes=7):
        super().__init__()
        self.image_backbone = image_backbone
        self.lmk_backbone = lmk_backbone
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

    def call(self, inputs, training=False):
        image_input, landmark_input = inputs
        img_feat = self.image_backbone(image_input, training=training)
        lmk_feat = self.lmk_backbone(landmark_input, training=training)
        fused = tf.concat([img_feat, lmk_feat], axis=-1)
        return self.classifier(fused, training=training)

model = LateFusionModel(resnet_backbone, mlp_backbone)

def preprocess_sample(sample):
    img = Image.open(sample['img']).convert('RGB').resize((112, 112))
    img_arr = np.array(img).astype(np.float32)
    img_arr = (img_arr - 127.5) / 127.5
    img_arr = np.transpose(img_arr, (2, 0, 1))  # (3, 112, 112)

    lmk = np.load(sample['npy']).astype(np.float32)
    lmk_flat = lmk.flatten()

    label = sample['label']
    return img_arr, lmk_flat, label

def data_generator(data_list):
    for sample in data_list:
        img, lmk, label = preprocess_sample(sample)
        yield (img, lmk), label

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_data),
    output_signature=(
        (tf.TensorSpec(shape=(3, 112, 112), dtype=tf.float32),
         tf.TensorSpec(shape=(1434,), dtype=tf.float32)),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
).batch(32).prefetch(tf.data.AUTOTUNE)

all_preds, all_labels, all_logits, all_features = [], [], [], []

for (img_batch, lmk_batch), labels in test_dataset:
    preds = model((img_batch, lmk_batch), training=False)

    img_feat = resnet_backbone(img_batch, training=False)
    lmk_feat = mlp_backbone(lmk_batch, training=False)
    fused = tf.concat([img_feat, lmk_feat], axis=-1)

    all_preds.extend(tf.argmax(preds, axis=1).numpy())
    all_labels.extend(labels.numpy())
    all_logits.extend(preds.numpy())
    all_features.extend(fused.numpy())

all_labels_np = np.array(all_labels)
all_preds_np = np.array(all_preds)
all_logits_np = np.array(all_logits)

acc = accuracy_score(all_labels_np, all_preds_np)

ce_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
cross_entropy = ce_loss_fn(all_labels_np, all_logits_np).numpy()

uniform_probs = np.full_like(all_logits_np, 1.0 / all_logits_np.shape[1])
kl_div_fn = tf.keras.losses.KLDivergence()
kl_div = kl_div_fn(uniform_probs, all_logits_np).numpy()

conf_matrix = confusion_matrix(all_labels_np, all_preds_np, labels=np.arange(len(class_names)))
per_class_acc = conf_matrix.diagonal() / np.maximum(conf_matrix.sum(axis=1), 1)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Emotion')
plt.ylabel('True Emotion')
plt.gca().invert_yaxis()
plt.show()

tsne = TSNE(n_components=2, random_state=42)
tsne_proj = tsne.fit_transform(np.array(all_features))

plt.figure(figsize=(8, 6))
scatter = plt.scatter(tsne_proj[:, 0], tsne_proj[:, 1], c=all_labels_np, cmap='tab10', alpha=0.6)
handles, _ = scatter.legend_elements()
plt.legend(handles, class_names, title='Emotions')
plt.title('t-SNE of Fused Features')
plt.show()

print(f"Top-1 Accuracy: {acc:.4f}")
print(f"Cross-Entropy Loss: {cross_entropy:.4f}")
print(f"KL Divergence (vs uniform): {kl_div:.4f}")
print("Per-class Accuracy:")
for i, a in enumerate(per_class_acc):
    print(f"  {class_names[i]}: {a:.4f}")

print("\nClassification Report:")
print(classification_report(all_labels_np, all_preds_np, target_names=class_names, digits=4))

print("\n Label distribution in test data:")
(unique, counts) = np.unique(all_labels_np, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  {class_names[u]} ({u}): {c} samples")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/facial_fuse/label-distribution-learning-fer-tf/pretrained/mlp_landmark_backbone.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)